In [14]:
import cv2
import easyocr
import re

# Charger l'image
image = cv2.imread('/Users/sebastianonise/Downloads/videoframe_37664.png')

# Lancer EasyOCR (français + anglais)
reader = easyocr.Reader(['fr', 'en'])

# Lire tous les textes de l'image
results = reader.readtext(image)

# Extraire les textes seulement
detected_texts = [text for (_, text, _) in results]

print(detected_texts)


# Appliquer un filtrage intelligent
km_candidates = []
for text in detected_texts:
    cleaned = text.replace(" ", "")
    if re.fullmatch(r"\d{4,6}(km)?", cleaned):
        km_candidates.append(cleaned)

print("Candidats kilométrage:", km_candidates)


['9537', 'n', '700962', 'km', '8', '0oo']
Candidats kilométrage: ['9537', '700962']


In [15]:
import easyocr
import cv2
import re

def extract_kilometrage_with_easyocr(image_path):
    reader = easyocr.Reader(['fr', 'en'], gpu=False)

    image = cv2.imread(image_path)
    image_resized = cv2.resize(image, (800, int(image.shape[0] * (800 / image.shape[1]))))
    height, width, _ = image_resized.shape

    results = reader.readtext(image_resized)
    candidates = []

    for (bbox, text, conf) in results:
        clean_text = text.replace(" ", "").lower()
        if not re.fullmatch(r"\d{4,6}(km)?", clean_text):
            continue

        # Position du texte (centre)
        (tl, tr, br, bl) = bbox
        center_x = int((tl[0] + tr[0] + br[0] + bl[0]) / 4)
        center_y = int((tl[1] + tr[1] + br[1] + bl[1]) / 4)

        score = 0
        if "km" in clean_text:
            score += 2
        if re.fullmatch(r"\d{5,6}(km)?", clean_text):
            score += 2
        if 0.3 * width < center_x < 0.7 * width:
            score += 1
        if 0.5 * height < center_y < 0.95 * height:
            score += 1
        value = int(re.sub(r"[^\d]", "", clean_text))
        if 10000 <= value <= 500000:
            score += 2

        candidates.append({
            "text": text,
            "score": score,
            "value": value,
            "center": (center_x, center_y),
            "bbox": bbox
        })

    if not candidates:
        return None, []

    best_candidate = max(candidates, key=lambda x: x["score"])
    return best_candidate["value"], candidates

# Exemple d'utilisation
if __name__ == "__main__":
    image_path = "/Users/sebastianonise/Downloads/videoframe_37664.png"
    km, all_candidates = extract_kilometrage_with_easyocr(image_path)
    print("Kilométrage détecté :", km)
    print("Tous les candidats :", all_candidates)


Using CPU. Note: This module is much faster with a GPU.


Kilométrage détecté : 9537
Tous les candidats : [{'text': '9537', 'score': 1, 'value': 9537, 'center': (479, 216), 'bbox': [[np.int32(423), np.int32(195)], [np.int32(535), np.int32(195)], [np.int32(535), np.int32(237)], [np.int32(423), np.int32(237)]]}]
